# Download OS Open Data

https://realpython.com/python-download-file-from-url/#saving-downloaded-content-to-a-file

https://www.dataquest.io/blog/python-api-tutorial/

In [ ]:
import requests
import os
import json
from zipfile import ZipFile
import pandas as pd

In [ ]:
# Get the current working directory
os.getcwd()

In [ ]:
# Add the main filepath
main_file_path = "//192.168.101.45/gis/_Projects/SpeciesDataPrep_20241209_48ad30"

# Location where data will be downloaded
download_location = main_file_path + "/commondata/userdata/Data_download/Downloads"

item_url_file = "OS-API_URL.csv"

In [ ]:
# Change the current working directory
os.chdir(download_location)

In [ ]:
os.getcwd()

The OS-API_URL.csv file contains the column 'URL' with the URLs of the datasets that you want to download.

In [ ]:
# Read the csv file
item_url_df = pd.read_csv(main_file_path + "/commondata/userdata/Data_download/Source_files/" + item_url_file)

In [ ]:
item_url_df.head()

In [ ]:
# Create a list of the item URLs
url_list = item_url_df['URL'].tolist()

In [ ]:
# Download the data
for url in url_list:
    response = requests.get(url)
    
    url_split = url.split('/downloads?')
    url_2 = url_split[0]
    response_2 = requests.get(url_2)
    
    #https://stackoverflow.com/questions/16877422/whats-the-best-way-to-parse-a-json-response-from-the-requests-library
    details = json.loads(response_2.text)
    
    if response.status_code == 200 and response_2.status_code == 200:
        if 'area=GB' in url:
            # https://www.w3schools.com/python/python_json.asp
            name = details['id'] + ".zip"
            
            with open(name, mode="wb") as file:
                file.write(response.content)
        else:
            # https://www.geeksforgeeks.org/python-extract-string-between-two-substrings/
            url_3 = url.replace('?area=','*')
            url_3 = url_3.replace('&format','*')
            url_split_2 = url_3.split('*')
            data_area = url_split_2[1]
            
            name = details['id'] + "_" + data_area + ".zip"
            
            with open(name, mode="wb") as file:
                file.write(response.content)

In [ ]:
# return all zip files at download_location as a list
zip_list = []

for file in os.listdir(download_location):
     # check the files which end in .zip
    if file.endswith(".zip"):
        # add them to zip_list
        zip_list.append(file)
        
zip_list

In [ ]:
# Remove '.zip' extension from list items
name_list = []

for file_name in zip_list:
    file_name_2 = file_name.replace(".zip", "")
    name_list.append(file_name_2)
    
name_list

In [ ]:
# Unzip all zip files in download_location
for (z_file, z_folder) in zip(zip_list, name_list):
    with ZipFile(z_file, 'r') as z_object:
        z_object.extractall(path=z_folder)

In [ ]:
# Delete zip fiels
for file in zip_list:
    os.remove(file)

In [ ]:
# magic command to clear all variables in the namespace
# https://community.esri.com/t5/python-documents/arcgis-notebooks-in-arcgis-pro-3-1-faq/ta-p/1261606/jump-to/first-unread-message
%reset -f